In [ ]:
from keras import models,Input
from keras.models import Sequential, Model
from keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten,MaxPooling2D, InputLayer
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Reshape, Permute, Multiply,GlobalMaxPooling2D,Concatenate
from keras.layers import Concatenate,Add,Layer
#from spp.SpatialPyramidPooling import SpatialPyramidPooling
from keras.utils.vis_utils import plot_model

In [ ]:
#from keras.engine.topology import Layer
import keras.backend as K


class SpatialPyramidPooling(Layer):
    """Spatial pyramid pooling layer for 2D inputs.
    See Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition,
    K. He, X. Zhang, S. Ren, J. Sun
    # Arguments
        pool_list: list of int
            List of pooling regions to use. The length of the list is the number of pooling regions,
            each int in the list is the number of regions in that pool. For example [1,2,4] would be 3
            regions with 1, 2x2 and 4x4 max pools, so 21 outputs per feature map
    # Input shape
        4D tensor with shape:
        `(samples, channels, rows, cols)` if dim_ordering='th'
        or 4D tensor with shape:
        `(samples, rows, cols, channels)` if dim_ordering='tf'.
    # Output shape
        2D tensor with shape:
        `(samples, channels * sum([i * i for i in pool_list])`
    """

    def __init__(self, pool_list, **kwargs):

        self.dim_ordering = K.image_dim_ordering()
        assert self.dim_ordering in {'tf', 'th'}, 'dim_ordering must be in {tf, th}'

        self.pool_list = pool_list

        self.num_outputs_per_channel = sum([i * i for i in pool_list])

        super(SpatialPyramidPooling, self).__init__(**kwargs)

    def build(self, input_shape):
        if self.dim_ordering == 'th':
            self.nb_channels = input_shape[1]
        elif self.dim_ordering == 'tf':
            self.nb_channels = input_shape[3]

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.nb_channels * self.num_outputs_per_channel)

    def get_config(self):
        config = {'pool_list': self.pool_list}
        base_config = super(SpatialPyramidPooling, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def call(self, x, mask=None):

        input_shape = K.shape(x)

        if self.dim_ordering == 'th':
            num_rows = input_shape[2]
            num_cols = input_shape[3]
        elif self.dim_ordering == 'tf':
            num_rows = input_shape[1]
            num_cols = input_shape[2]

        row_length = [K.cast(num_rows, 'float32') / i for i in self.pool_list]
        col_length = [K.cast(num_cols, 'float32') / i for i in self.pool_list]

        outputs = []

        if self.dim_ordering == 'th':
            for pool_num, num_pool_regions in enumerate(self.pool_list):
                for jy in range(num_pool_regions):
                    for ix in range(num_pool_regions):
                        x1 = ix * col_length[pool_num]
                        x2 = ix * col_length[pool_num] + col_length[pool_num]
                        y1 = jy * row_length[pool_num]
                        y2 = jy * row_length[pool_num] + row_length[pool_num]

                        x1 = K.cast(K.round(x1), 'int32')
                        x2 = K.cast(K.round(x2), 'int32')
                        y1 = K.cast(K.round(y1), 'int32')
                        y2 = K.cast(K.round(y2), 'int32')
                        new_shape = [input_shape[0], input_shape[1],
                                     y2 - y1, x2 - x1]
                        x_crop = x[:, :, y1:y2, x1:x2]
                        xm = K.reshape(x_crop, new_shape)
                        pooled_val = K.max(xm, axis=(2, 3))
                        outputs.append(pooled_val)

        elif self.dim_ordering == 'tf':
            for pool_num, num_pool_regions in enumerate(self.pool_list):
                for jy in range(num_pool_regions):
                    for ix in range(num_pool_regions):
                        x1 = ix * col_length[pool_num]
                        x2 = ix * col_length[pool_num] + col_length[pool_num]
                        y1 = jy * row_length[pool_num]
                        y2 = jy * row_length[pool_num] + row_length[pool_num]

                        x1 = K.cast(K.round(x1), 'int32')
                        x2 = K.cast(K.round(x2), 'int32')
                        y1 = K.cast(K.round(y1), 'int32')
                        y2 = K.cast(K.round(y2), 'int32')

                        new_shape = [input_shape[0], y2 - y1,
                                     x2 - x1, input_shape[3]]

                        x_crop = x[:, y1:y2, x1:x2, :]
                        xm = K.reshape(x_crop, new_shape)
                        pooled_val = K.max(xm, axis=(1, 2))
                        outputs.append(pooled_val)

        if self.dim_ordering == 'th':
            outputs = K.concatenate(outputs)
        elif self.dim_ordering == 'tf':
            #outputs = K.concatenate(outputs,axis = 1)
            outputs = K.concatenate(outputs)
            #outputs = K.reshape(outputs,(len(self.pool_list),self.num_outputs_per_channel,input_shape[0],input_shape[1]))
            #outputs = K.permute_dimensions(outputs,(3,1,0,2))
            #outputs = K.reshape(outputs,(input_shape[0], self.num_outputs_per_channel * self.nb_channels))

        return outputs

In [ ]:

from keras.layers.merge import concatenate
#CNN
#Module1
inputLay = Input(shape=(200,200,1))
x = Conv2D(32, (3,3), padding='same')(inputLay)
x = Activation('relu')(x)
x = Conv2D(32, (3,3), padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.3)(x)

x = Conv2D(64, (3,3), padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3,3), padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.3)(x)

x = Conv2D(128, (3,3), padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(128, (3,3), padding='same')(x)

x = BatchNormalization()(x)

#SE1 Block
#inio Squeeze 
y1 = GlobalAveragePooling2D()(x)
y2 = GlobalMaxPooling2D()(x)
#y= Concatenate(axis=1)([y1,y2])
y=Add()([y1,y2])
y = Reshape((1,1,128))(y)
y = Flatten()(y)
#FC layer1
y = Dense(units=8,activation="relu")(y)

#Excitation
y = Dense(units=128,activation="sigmoid")(y)

x = Multiply()([x, y])






#Concatenation of two modules



x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='feature_layer')(x)

x = Flatten()(x)
x = Dense(256)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)
x = Dense(23)(x)
x = BatchNormalization()(x)
outputLay = Activation('softmax')(x)

model = Model(inputLay, outputLay)

model.summary()
#plot_model(model, to_file='model.png')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200, 200, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 200, 200, 32  320         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 activation (Activation)        (None, 200, 200, 32  0           ['conv2d[0][0]']                 
                                )                                                             

In [ ]:
#Module 2
#dim_ordering = K.image_dim_ordering()
inputLay = Input(shape=(200,200,1))
x = Conv2D(32, (3,3), padding='same')(inputLay)
x = Activation('relu')(x)
x = Conv2D(32, (3,3), padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.3)(x)

x = Conv2D(64, (3,3), padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3,3), padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.3)(x)

x = Conv2D(128, (3,3), padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(128, (3,3), padding='same')(x)

x = BatchNormalization()(x)

#SE2 Block
#inio Squeeze 

y = SpatialPyramidPooling([1,2,4])
y = Reshape((1,1,128))(y)
y = Flatten()(y)
#FC layer1
y = Dense(units=8,activation="relu")(y)

#Excitation
y = Dense(units=128,activation="sigmoid")(y)

x = Multiply()([x, y])
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='feature_layer')(x)

x = Flatten()(x)
x = Dense(256)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)
x = Dense(23)(x)
x = BatchNormalization()(x)
outputLay = Activation('softmax')(x)

model = Model(inputLay, outputLay)

model.summary()
#plot_model(model, to_file='model.png')

AttributeError: ignored